### Исследование по оценке работы алгоритмов линейной, полиномиальной регрессий для данных, представимых в виде полиномиальной или экпоненциальной функций

---
#### Генерация данных

Данные будем генерировать при помощи встроенного в numpy класса `Polynomial`

Для полиномов:
```cpp
p = Polynomial(coeffs[::-1], domain=domain, window=domain)
xdata['deg 1'], ydata['poly'] = p.linspace(N)
```

Для экпоненциальных функций:
```python
def exp_func(p: Polynomial, c: float, N: int = N) -> np.ndarray:
    return np.exp(p.linspace(N)[1]) + c
```

Генериуем <b>N</b> равномерно расположенных на области определения точек

Для правдоподобности добавляем шум в сгенерированные данные:
```cpp
noize = np.random.normal(size=N, scale=np.mean(abs(ydata['poly']))/4)
ydata_noized = ydata['poly'] + noize
```

---
#### Обучение

Для обучения используем линейную регрессию, `PolynomialFeatures`, `FunctionTransformer` из пакета `sklearn`

Для полиномов 1 степени:
```cpp
reg[i] = LinearRegression().fit(xdata[i], ydata[k])
```

Для полиномов степеней 2 - 4:
```cpp
degs[i] = PolynomialFeatures(degree=j)
xdata[i] = degs[i].fit_transform(xdata['deg 1'])
reg[i] = LinearRegression().fit(xdata[i], ydata[k])
```

Для экспоненциальных функций
```cpp
xy = pd.DataFrame({'x': xdata['deg 1'][:, 0], 'y': ydata_noized})
xy = xy[xy['y'] > 0]
trans = FunctionTransformer(np.log, validate=True)
ydata['exp'] = trans.fit_transform(np.array(xy['y']).reshape(-1, 1))
xdata['exp'] = np.array(xy['x']).reshape(-1, 1)
reg['exp'] = LinearRegression().fit(xdata['exp'], ydata['exp'])
new_y['exp'] = np.exp(reg['exp'].predict(new_x['exp']))[:, 0]
```

---
#### Оценка работы

Для оценки работы используем коэффициент детерминации

$R^2=1-\frac{V(y|x)}{V(y)}=1-\frac{\sigma^{2}}{\sigma_{y}^{2}}$

Где $V(y|x)=\sigma^2$ - дисперсия случайной ошибки модели

Получаем его при помощи функции из библиотеки `sklearn`
```cpp
r2[i] = r2_score(ydata_noized, reg[i].predict(xdata[i]))
```

---
#### Тесты

##### Первый тест - полином 1 степени $\frac{-x}{2} + \footnotesize{3}$

<img src="images/1_degree_poly_1_degree_fit.png" alt="image" width="49%"/>
<img src="images/1_degree_poly_2_degree_fit.png" alt="image" width="49%"/>
<img src="images/1_degree_poly_3_degree_fit.png" alt="image" width="49%"/>
<img src="images/1_degree_poly_4_degree_fit.png" alt="image" width="49%"/>
<img src="images/1_degree_poly_exp_fit.png" alt="image" width="49%"/>
<img src="images/1_degree_poly_all_together.png" alt="image" width="49%"/>

degree|$R^2$|
-|:----:|
1|0.9426711700857244|
2|0.9426711700857245|
3|0.9426711700857244|
4|0.9426711700857136|
exp|0.0850200696753181|

##### Второй тест - полином 2 степени $\frac{x^2}{20}-\footnotesize{x}$

<img src="images/2_degree_poly_1_degree_fit.png" alt="image" width="49%"/>
<img src="images/2_degree_poly_2_degree_fit.png" alt="image" width="49%"/>
<img src="images/2_degree_poly_3_degree_fit.png" alt="image" width="49%"/>
<img src="images/2_degree_poly_4_degree_fit.png" alt="image" width="49%"/>
<img src="images/2_degree_poly_exp_fit.png" alt="image" width="49%"/>
<img src="images/2_degree_poly_all_together.png" alt="image" width="49%"/>

 degree|$R^2$|
-|:----:|
1|0.901020005632132
2|0.968224851751847
3|0.968224851751847
4|0.968224851751846
exp|0.669329407932745

##### Третий тест - полином 3 степени $\frac{x^3}{20}-\frac{3x^2}{50}-\footnotesize{4x}$

<img src="images/3_degree_poly_1_degree_fit.png" alt="image" width="49%"/>
<img src="images/3_degree_poly_2_degree_fit.png" alt="image" width="49%"/>
<img src="images/3_degree_poly_3_degree_fit.png" alt="image" width="49%"/>
<img src="images/3_degree_poly_4_degree_fit.png" alt="image" width="49%"/>
<img src="images/3_degree_poly_exp_fit.png" alt="image" width="49%"/>
<img src="images/3_degree_poly_all_together.png" alt="image" width="49%"/>

 degree|$R^2$|
-|:----:|
1|0.104663666506075
2|0.127916030797361
3|0.961163381723431
4|0.961163381723420
exp|0.087462128260766

##### Четвертый тест - полином 4 степени $\frac{x^4}{160} + \frac{x^3}{20} -\frac{3x^2}{5} -\footnotesize{x}$

<img src="images/4_degree_poly_1_degree_fit.png" alt="image" width="49%"/>
<img src="images/4_degree_poly_2_degree_fit.png" alt="image" width="49%"/>
<img src="images/4_degree_poly_3_degree_fit.png" alt="image" width="49%"/>
<img src="images/4_degree_poly_4_degree_fit.png" alt="image" width="49%"/>
<img src="images/4_degree_poly_exp_fit.png" alt="image" width="49%"/>
<img src="images/4_degree_poly_all_together.png" alt="image" width="49%"/>

 degree|$R^2$|
-|:----:|
1|0.5784239365423052
2|0.5786838634695093
3|0.8578833661112893
4|0.9648199060313363
exp|0.5178778886773503

##### Пятый тест - экпоненциальный полином $e^{\frac{x}{2}}$

<img src="images/exp_poly_1_degree_fit.png" alt="image" width="49%"/>
<img src="images/exp_poly_2_degree_fit.png" alt="image" width="49%"/>
<img src="images/exp_poly_3_degree_fit.png" alt="image" width="49%"/>
<img src="images/exp_poly_4_degree_fit.png" alt="image" width="49%"/>
<img src="images/exp_poly_exp_fit.png" alt="image" width="49%"/>
<img src="images/exp_poly_all_together.png" alt="image" width="49%"/>

 degree|$R^2$|
-|:----:|
1|0.1286213775005266
2|0.3070932667583635
3|0.4969797279171785
4|0.6664289716548277
exp|0.8239630792765567

---
#### Выводы

1. Регрессии высших степеней не переобучаются на данных со справедливым уровнем шума и выдают такие же результаты как и регрессия степени равной степени самой функции

2. Линейная регрессия спокойно определяет полиномы любых степеней, если уровень шума в пределах справедливого